# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

# Pre-Requisite: Make Sure You Created an Athena Table for Both TSV and Parquet in Previous Notebooks

In [ ]:
## Not sure how important this notebook is to the overall workflow

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


In [5]:
%store -r ingest_create_athena_table_parquet_passed

In [6]:
try:
    ingest_create_athena_table_parquet_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [7]:
print(ingest_create_athena_table_parquet_passed)

True


In [8]:
if not ingest_create_athena_table_parquet_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.') 
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


In [9]:
import boto3
import sagemaker

session = boto3.session.Session()
region_name = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [10]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

_dataset (bool)_ – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [11]:
filter = lambda x: True if x["product_category"] == "Digital_Software" else False

In [12]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                        columns=['star_rating', 'product_category', 'review_body'],
                                        partition_filter=filter,
                                        dataset=True)
df_parquet_results.shape

(102084, 3)

In [13]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,4,So far so good,Digital_Software
1,3,Needs a little more work.....,Digital_Software
2,1,Please cancel.,Digital_Software
3,5,Works as Expected!,Digital_Software
4,4,I've had Webroot for a few years. It expired a...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [14]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                # filters=[("product_category", "=", "Digital_Software")],
                                partition_filter=filter,
                                dataset=True,
                                chunked=True)

In [15]:
print(next(chunk_iter))

       star_rating                                        review_body  \
0                4                                     So far so good   
1                3                      Needs a little more work.....   
2                1                                     Please cancel.   
3                5                                 Works as Expected!   
4                4  I've had Webroot for a few years. It expired a...   
...            ...                                                ...   
67336            4  File  electron federal return is free. But you...   
67337            3  The download went fine but the install hung on...   
67338            3  I've read some complaints about people who dow...   
67339            4  Very good product. I have been using Tax Cut f...   
67340            2  Okay first off the software works pretty good,...   

       product_category  
0      Digital_Software  
1      Digital_Software  
2      Digital_Software  
3      Digital_Soft

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [16]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [17]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [18]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database=database_name
)

CPU times: user 461 ms, sys: 36.6 ms, total: 497 ms
Wall time: 8.16 s


In [19]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,15257192,R204T21JTCZ8XG,B00E7XA7KY,189774198,Quickbooks Pro,1,1,1,N,N,Chase bank Web Connect issues,Major problems with Chase bank intergration. C...,2014,2014-02-16,Digital_Software
1,US,50690611,R2PCSOYZZ5IAL1,B00FFINUJK,866839083,"TurboTax Home and Business Fed, Efile and Stat...",5,2,2,N,Y,Long time user: It does what it's supposed to ...,Initial note: This is a review of the download...,2014,2014-02-16,Digital_Software
2,US,2481589,R3N8V9RJWPDXOA,B00B1TGHXS,954368001,Microsoft Word 2013 (1PC/1User),1,0,0,N,Y,was not able to download this program. i have ...,please refund for this program. I can not down...,2014,2014-02-16,Digital_Software
3,US,42625099,R2LV5O1P2A5GFX,B00F8LJU9S,627104528,Adobe Photoshop Elements 12,5,0,0,N,Y,Easy to use,Super easy to download and great software prog...,2014,2014-02-16,Digital_Software
4,US,51561267,R2D8O9VZOL67J6,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,5,0,1,N,Y,Long Time Norton Fan,I've been using Norton Internet Security for y...,2014,2014-02-16,Digital_Software


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [20]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 282 ms, sys: 20 ms, total: 302 ms
Wall time: 6.76 s


In [21]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    15257192  R204T21JTCZ8XG  B00E7XA7KY      189774198   
1             US    50690611  R2PCSOYZZ5IAL1  B00FFINUJK      866839083   
2             US     2481589  R3N8V9RJWPDXOA  B00B1TGHXS      954368001   
3             US    42625099  R2LV5O1P2A5GFX  B00F8LJU9S      627104528   
4             US    51561267  R2D8O9VZOL67J6  B00FGDEPDY      991059534   
...          ...         ...             ...         ...            ...   
1019          US    42710505  R1BPHQB726D55L  B00FFINOWS      875090538   
1020          US    47659476  R14M5Z76K6HE7O  B00H9A60O4      608720080   
1021          US    18966808  R19TXQGIXPUV5B  B00H9A60O4      608720080   
1022          US    21334179  R2VUGMBR883BEE  B00FFINOWS      875090538   
1023          US    14843482   RKSQEVQK2PO2W  B00FGDDTSQ      672725528   

                                          product_title  star_rating  \
0                          

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>